### Explicación del Código

El código en `tets_historias.ipynb` procesa un archivo CSV con información de personas desaparecidas y genera múltiples variaciones narrativas para cada caso utilizando prompts predefinidos. Además, guarda los resultados en un archivo CSV y genera un archivo HTML para visualización. Realiza las siguientes tareas:

1. **Selección de Archivo CSV**:
   - Busca archivos CSV en la carpeta `input_folder`.
   - Si hay múltiples archivos, solicita al usuario que seleccione uno.

2. **Filtrado de Casos**:
   - Procesa solo los casos donde `condicion_localizacion` es "NO APLICA" y la columna `descripcion_desaparicion` no está vacía.

3. **Generación de Historias**:
   - Utiliza 9 estilos narrativos predefinidos (e.g., melancólico, poético, estilo Rulfo) para generar historias breves basadas en los datos de cada caso.
   - Las historias se generan mediante la API de DeepSeek.

4. **Exportación de Resultados**:
   - Guarda los casos procesados con las historias generadas en un archivo CSV en la carpeta `output`.
   - Genera un archivo HTML con tarjetas visuales que muestran las historias generadas para cada caso.

5. **Carpetas y Archivos Relevantes**:
   - **Entrada**: Archivos CSV en `input_folder`.
   - **Salida**:
     - CSV: `output/processed_cases_multi_prompt_<timestamp>.csv`.
     - HTML: `output/visualizacion_casos_<timestamp>.html`.

In [ ]:
# Instalar dependencias necesarias
!pip install openai pandas

In [ ]:
import os
import csv
import random
import logging
import time
from datetime import datetime
from openai import OpenAI # Mantenemos la librería aunque usemos Deepseek
import html # Para escapar caracteres HTML

# --- Configuración de Prompts ---
# (Prompts con restricciones de longitud y anonimización reforzada - SIN CAMBIOS DESDE LA VERSIÓN ANTERIOR)
PROMPT_STYLES = [
    {
        'name': 'Original_Melancolico',
        'system': (
            "Eres un asistente útil que reescribe descripciones de desaparición como historias "
            "muy breves en español. Inspiras melancolía pero no drama. Narras una desaparición de forma concisa."
        ),
        'user_template': (
            "Recrea esta descripción como una historia MUY BREVE (máximo 4-5 líneas). No agregues título. "
            "Menciona fecha/hora si es posible. Narra la desaparición de {nombre_completo} ({edad_momento_desaparicion} años). "
            "IMPORTANTE: Elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse (en Sentence case). "
            "La historia resultante NO debe exceder las 5 líneas de texto. Descripción base: {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Estilo_Rulfo',
        'system': (
            "Eres un escritor que emula el estilo conciso y evocador de Juan Rulfo. Usas frases cortas, tono "
            "seco, casi susurrante. Describes la ausencia."
        ),
        'user_template': (
            "Como un eco en Comala, narra en estilo Rulfiano MUY BREVE (máximo 4-5 líneas) la ausencia de {nombre_completo} "
            "({edad_momento_desaparicion} años). Menciona fecha/hora si se intuye. Que quede la sensación del vacío. "
            "IMPORTANTE: Elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse. "
            "La historia resultante NO debe exceder las 5 líneas de texto. Descripción base: {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Haiku_Desaparicion',
        'system': (
            "Eres un poeta que compone haikus (tres versos, 5-7-5 sílabas) en español sobre "
            "momentos específicos y sentimientos."
        ),
        'user_template': (
            "Crea un haiku en español que capture la esencia del momento de la desaparición de {nombre_completo} "
            "({edad_momento_desaparicion} años), basándote en la fecha/hora si es posible. "
            "IMPORTANTE: NO incluyas NINGÚN domicilio, calle, comercio, placa, ni nombres de otras personas. "
            "El nombre {nombre_completo} puede omitirse si el sentimiento de ausencia es claro. "
            "Descripción para inspirarte (no incluyas detalles específicos de ella): {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Narrativa_Poetica',
        'system': (
            "Eres un narrador con una voz poética concisa. Usas metáforas sutiles y lenguaje "
            "evocador para describir eventos tristes sin melodrama, en pocas líneas."
        ),
        'user_template': (
            "Narra poéticamente, en un párrafo MUY CORTO (máximo 4-5 líneas), la desaparición de {nombre_completo} "
            "({edad_momento_desaparicion} años). Enfócate en la atmósfera del momento (usa fecha/hora si aplica). "
            "IMPORTANTE: Elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse. "
            "La historia resultante NO debe exceder las 5 líneas de texto. Descripción base: {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Cronica_Breve',
        'system': (
            "Eres un cronista que relata hechos de forma muy concisa y objetiva, con un leve toque humano."
        ),
        'user_template': (
            "Redacta una crónica EXTREMADAMENTE BREVE (2-4 frases, máximo 5 líneas) sobre la desaparición de {nombre_completo} "
            "({edad_momento_desaparicion} años), incluyendo fecha/hora si se mencionan. Sé objetivo pero humano. "
            "IMPORTANTE: Omite y elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse. "
            "La historia resultante NO debe exceder las 5 líneas de texto. Hechos base: {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Voz_Esperanza',
        'system': (
            "Eres una voz concisa que, incluso al narrar una desaparición, deja una mínima puerta abierta a la esperanza, "
            "enfocándose en la persona y el recuerdo vivo."
        ),
        'user_template': (
            "Cuenta brevemente (máximo 4-5 líneas) la historia de la desaparición de {nombre_completo} ({edad_momento_desaparicion} años), "
            "mencionando fecha/hora si es posible. Narra la ausencia, pero enfócate en quién era {nombre_completo} o en la búsqueda. "
            "IMPORTANTE: Elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse. "
            "La historia resultante NO debe exceder las 5 líneas de texto. Descripción: {descripcion_desaparicion}."
        )
    },
    {
        'name': 'Estilo_Noticia_Antigua',
        'system': (
             "Eres un redactor de noticias de un periódico antiguo. Usas lenguaje formal, directo y muy conciso."
        ),
        'user_template': (
            "Redacta como una MUY BREVE nota de periódico antiguo (máximo 4-5 líneas) la noticia de la desaparición de {nombre_completo}, "
            "de {edad_momento_desaparicion} años. Incluye fecha y hora si se conocen. Sé factual y conciso. "
            "IMPORTANTE: Omite y elimina por completo cualquier domicilio, número de casa, nombre de calle, nombre de comercio, placa de vehículo o dato sensible similar. "
            "Anonimiza nombres de terceras personas. El nombre {nombre_completo} y su edad SÍ deben mencionarse. "
            "La historia resultante NO debe exceder las 5 líneas de texto. Basado en: {descripcion_desaparicion}."
         )
    },
    {
        'name': 'Microcuento_Fantastico',
        'system': (
             "Eres un escritor de microcuentos con un toque fantástico o surrealista. Insinúas más de lo que cuentas, de forma muy breve."
        ),
        'user_template': (
            "Convierte la desaparición de {nombre_completo} ({edad_momento_desaparicion} años) en un microcuento "
            "(máximo 4-5 líneas) con un elemento sutilmente fantástico o inexplicable. Menciona fecha/hora como parte del misterio si es posible. "
            "Sé EXTREMADAMENTE breve. "
            "IMPORTANTE: No incluyas NINGÚN domicilio, calle, comercio, placa ni nombres de otras personas. Evita datos reales identificables. "
            "El protagonista {nombre_completo} y su edad SÍ deben mencionarse (o insinuarse). "
            "La historia resultante NO debe exceder las 5 líneas de texto. Descripción original (solo como inspiración): {descripcion_desaparicion}."
         )
    }
]


# Configuración del cliente de DeepSeek (¡Usa tu API Key real!)
try:
    # Poner tu API Key real aquí
    client = OpenAI(api_key="sk-xxx", base_url="https://api.deepseek.com")
except Exception as e:
    logging.error(f"Error al inicializar el cliente de DeepSeek: {e}")
    client = None

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def generate_story_variation(case, prompt_style, client_instance):
    # (Función SIN CAMBIOS DESDE LA VERSIÓN ANTERIOR)
    if not client_instance:
        return "Error: Cliente DeepSeek no inicializado."
    try:
        nombre = case.get('nombre_completo', 'Nombre Desconocido')
        edad = case.get('edad_momento_desaparicion', 'Edad Desconocida')
        descripcion = case.get('descripcion_desaparicion', 'Sin descripción.')

        if not descripcion.strip():
             descripcion = "Detalles no proporcionados."
             logging.warning(f"Descripción vacía para {nombre}, usando texto alternativo.")

        user_content = prompt_style['user_template'].format(
            nombre_completo=nombre,
            edad_momento_desaparicion=edad,
            descripcion_desaparicion=descripcion
        )

        logging.debug(f"Enviando caso a DeepSeek con estilo '{prompt_style['name']}': Nombre={nombre}")
        response = client_instance.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": prompt_style['system']},
                {"role": "user", "content": user_content},
            ],
        )
        rewritten_story = response.choices[0].message.content
        logging.debug(f"Respuesta recibida de DeepSeek para '{prompt_style['name']}': {rewritten_story[:100]}...")
        time.sleep(1.5)
        return rewritten_story.strip()

    except KeyError as e:
         logging.error(f"Error de clave al formatear prompt para estilo '{prompt_style['name']}': Falta la clave {e} en el caso {case.get('nombre_completo')}")
         return f"Error Interno (Falta dato: {e})"
    except Exception as e:
        logging.error(f"Error comunicándose con DeepSeek para estilo '{prompt_style['name']}': {e}")
        time.sleep(5)
        return f"Error DeepSeek ({prompt_style['name']}): {e}"


# --- Función generate_html_output MODIFICADA ---
def generate_html_output(processed_cases, output_folder, timestamp):
    """Generates an HTML file using Bootstrap Cards for visualization."""
    if not processed_cases:
        logging.warning("No hay casos procesados para generar HTML.")
        return

    html_filename = os.path.join(output_folder, f"visualizacion_casos_{timestamp}.html")
    # Define qué columnas originales quieres mostrar en la cabecera de cada tarjeta
    original_columns_to_show = ['nombre_completo', 'edad_momento_desaparicion', 'sexo', 'fecha_desaparicion'] # Ajusta según tus columnas

    try:
        with open(html_filename, 'w', encoding='utf-8') as f:
            # Inicio del HTML con Bootstrap 5 CDN
            f.write(f"""<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Visualización de Casos Procesados - {timestamp}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-QWTKZyjpPEjISv5WaRU9OFeRpok6YctnYmDr5pNlyT2bRjXh0JMhjY6hW+ALEwIH" crossorigin="anonymous">
    <style>
        body {{ padding: 20px; background-color: #e9ecef; }} /* Fondo ligeramente gris */
        .card {{ margin-bottom: 1.5rem; border: none; box-shadow: 0 0.125rem 0.25rem rgba(0, 0, 0, 0.075); }} /* Sombra sutil */
        .card-header {{ background-color: #343a40; color: white; font-weight: bold; }} /* Cabecera oscura */
        .card-body h5 {{ color: #0056b3; margin-top: 1rem; margin-bottom: 0.5rem; }} /* Título de estilo azul */
        .card-body h5:first-of-type {{ margin-top: 0; }} /* Sin margen superior para el primer título */
        pre {{
            white-space: pre-wrap;
            word-wrap: break-word;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; /* Fuente más legible */
            font-size: 0.95em;
            background-color: #f8f9fa; /* Fondo claro para texto */
            padding: 10px;
            border-radius: 4px;
            margin-bottom: 1rem; /* Espacio después de cada historia */
            line-height: 1.6;
            border: 1px solid #dee2e6; /* Borde sutil */
        }}
        .case-info p {{ margin-bottom: 0.25rem; }} /* Menos espacio entre datos del caso */
    </style>
</head>
<body>
    <div class="container">
        <h1 class="my-4 text-center">Visualización Comparativa de Estilos Narrativos</h1>
        <p class="text-center text-muted mb-5">Generado el: {timestamp}</p>

        {f""}

""")

            # --- Bucle Principal por Caso ---
            for i, case in enumerate(processed_cases):
                f.write(f'        \n')
                f.write('        <div class="card">\n')

                # --- Cabecera de la Tarjeta (Datos del Caso) ---
                nombre_completo = html.escape(str(case.get('nombre_completo', 'N/A')))
                f.write(f'          <div class="card-header">Caso: {nombre_completo}</div>\n')
                f.write('           <div class="card-body">\n')
                f.write('               <div class="case-info mb-3">\n') # Contenedor para datos básicos
                for col_name in original_columns_to_show:
                    if col_name == 'nombre_completo': continue # Ya está en el header
                    header_text = col_name.replace('_', ' ').title()
                    cell_data = case.get(col_name, 'N/A')
                    f.write(f"                  <p><strong>{html.escape(header_text)}:</strong> {html.escape(str(cell_data))}</p>\n")
                f.write('               </div>\n')
                f.write('               <hr>\n') # Separador visual

                # --- Cuerpo de la Tarjeta (Historias Generadas) ---
                for style in PROMPT_STYLES:
                    style_name_clean = style['name'].replace('_', ' ')
                    story_column_name = f"historia_{style['name']}"
                    story_data = case.get(story_column_name, 'Error/No generado')

                    # Escribir nombre del estilo y la historia en <pre>
                    f.write(f"                  <h5>{html.escape(style_name_clean)}</h5>\n")
                    f.write(f'                  <pre>{html.escape(story_data)}</pre>\n')

                f.write('            </div> \n')
                f.write('        </div> \n')
                f.write(f'        \n\n')

            # Fin del contenedor y del HTML
            f.write("""
    </div> <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-YvpcrYf0tY3lHB60NNkmXc5s9fDVZLESaAA55NDzOxhy9GkcIdslK1eN7N6jIeHz" crossorigin="anonymous"></script>
</body>
</html>
""")
        logging.info(f"Archivo HTML de visualización (formato tarjetas) guardado en: {html_filename}")

    except Exception as e:
        logging.error(f"Error al generar el archivo HTML con formato tarjetas: {e}", exc_info=True)


def process_csv(input_file, output_folder, num_cases, client_instance):
    # (Función SIN CAMBIOS DESDE LA VERSIÓN ANTERIOR, solo llama a la nueva generate_html_output)
    if not client_instance:
        logging.error("El cliente DeepSeek no está disponible. Abortando proceso.")
        return

    processed_cases = []
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    try:
        all_cases_raw = []
        try:
            with open(input_file, 'r', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                if not reader.fieldnames:
                    logging.error(f"Error: El archivo CSV '{input_file}' está vacío o no tiene cabeceras.")
                    return
                required_cols = ['condicion_localizacion', 'descripcion_desaparicion', 'nombre_completo', 'edad_momento_desaparicion']
                missing_cols = [col for col in required_cols if col not in reader.fieldnames]
                if missing_cols:
                    logging.error(f"Error: Faltan columnas requeridas en el CSV: {', '.join(missing_cols)}")
                    return

                all_cases_raw = list(reader)
                logging.info(f"Leído {len(all_cases_raw)} casos del archivo: {input_file}")
        except FileNotFoundError:
            logging.error(f"Error Crítico: No se encontró el archivo de entrada: {input_file}")
            return
        except Exception as e:
            logging.error(f"Error Crítico al leer el archivo CSV '{input_file}': {e}", exc_info=True)
            return

        filtered_cases = []
        for i, row in enumerate(all_cases_raw):
            condicion = row.get('condicion_localizacion', '').strip().upper()
            descripcion = row.get('descripcion_desaparicion', '').strip()
            if condicion == "NO APLICA" and descripcion:
                filtered_cases.append(row)
            elif condicion != "NO APLICA":
                logging.debug(f"Caso {i+1} omitido (condición: '{condicion}')")
            elif not descripcion:
                 logging.debug(f"Caso {i+1} omitido (descripción vacía)")

        logging.info(f"Casos que cumplen criterio ('NO APLICA' y tienen descripción): {len(filtered_cases)}")

        if not filtered_cases:
             logging.warning("No se encontraron casos que cumplan los criterios para procesar.")
             return

        if len(filtered_cases) < num_cases:
            logging.warning(f"Solo {len(filtered_cases)} casos cumplen criterios, seleccionando todos.")
            selected_cases = filtered_cases
        else:
            selected_cases = random.sample(filtered_cases, num_cases)
        logging.info(f"Seleccionados {len(selected_cases)} casos para procesar.")

        total_api_calls = len(selected_cases) * len(PROMPT_STYLES)
        logging.info(f"Se realizarán aproximadamente {total_api_calls} llamadas a la API de DeepSeek.")

        for i, case in enumerate(selected_cases):
            case_name_log = case.get('nombre_completo', f'Caso índice {i}')
            logging.info(f"Procesando caso {i+1}/{len(selected_cases)}: {case_name_log}")
            processed_case_data = {k: v for k, v in case.items() if k != 'descripcion_desaparicion'}

            for style in PROMPT_STYLES:
                column_name = f"historia_{style['name']}"
                story_variation = generate_story_variation(case, style, client_instance)
                processed_case_data[column_name] = story_variation

            processed_cases.append(processed_case_data)

        if not processed_cases:
            logging.warning("No se procesó ningún caso exitosamente.")
            return

        # --- GUARDAR CSV ---
        os.makedirs(output_folder, exist_ok=True)
        output_csv_file = os.path.join(output_folder, f"processed_cases_multi_prompt_{timestamp}.csv")

        if processed_cases:
             fieldnames = list(processed_cases[0].keys())
             try:
                 with open(output_csv_file, 'w', encoding='utf-8', newline='') as csvfile:
                     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                     writer.writeheader()
                     writer.writerows(processed_cases)
                 logging.info(f"Archivo CSV procesado con {len(PROMPT_STYLES)} estilos y guardado en: {output_csv_file}")
             except IOError as e:
                  logging.error(f"Error al escribir el archivo CSV '{output_csv_file}': {e}")
             except Exception as e:
                  logging.error(f"Error inesperado al guardar el CSV: {e}", exc_info=True)
        else:
             logging.warning("No hay datos procesados para guardar en CSV.")

        # --- GENERAR HTML (ahora con la nueva función de tarjetas) ---
        if processed_cases:
            generate_html_output(processed_cases, output_folder, timestamp)
        else:
             logging.warning("No se generará HTML porque no hubo casos procesados exitosamente.")

    except Exception as e:
        logging.error(f"Error inesperado durante el procesamiento principal: {e}", exc_info=True)


# --- get_input_file y bloque __main__ SIN CAMBIOS DESDE LA VERSIÓN ANTERIOR ---

def get_input_file(input_folder):
    """Ensure input folder exists, prompt user for file selection if needed."""
    try:
        os.makedirs(input_folder, exist_ok=True)
        csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv') and os.path.isfile(os.path.join(input_folder, f))]

        if not csv_files:
            print(f"No se encontraron archivos CSV en la carpeta '{input_folder}'.")
            print("Por favor, coloca tus archivos CSV en esta carpeta y ejecuta el script de nuevo.")
            return None
        elif len(csv_files) == 1:
            selected_file = os.path.join(input_folder, csv_files[0])
            print(f"Archivo CSV encontrado y seleccionado automáticamente: {csv_files[0]}")
            return selected_file
        else:
            print("Se encontraron múltiples archivos CSV. Por favor, selecciona uno:")
            for i, file in enumerate(csv_files, 1):
                print(f"{i}. {file}")
            while True:
                try:
                    choice = int(input("Ingresa el número del archivo a usar: "))
                    if 1 <= choice <= len(csv_files):
                        selected_file = os.path.join(input_folder, csv_files[choice - 1])
                        print(f"Archivo seleccionado: {csv_files[choice - 1]}")
                        return selected_file
                    else:
                        print("Opción inválida. Intenta de nuevo.")
                except ValueError:
                    print("Entrada inválida. Por favor, ingresa un número.")
    except Exception as e:
        logging.error(f"Error al buscar o seleccionar el archivo de entrada: {e}")
        return None

if __name__ == "__main__":
    input_folder = 'input_folder'
    output_folder = 'output'

    if not client:
        logging.error("No se pudo inicializar el cliente de DeepSeek. Verifica tu API key y la conexión. El script no puede continuar.")
    else:
        input_csv = get_input_file(input_folder)
        if input_csv:
            while True:
                try:
                    num_styles = len(PROMPT_STYLES)
                    num_cases_str = input(f"Hay {num_styles} estilos de prompt definidos.\nIngresa el número de casos a procesar (cada caso generará {num_styles} historias): ")
                    num_cases = int(num_cases_str)
                    if num_cases > 0:
                        process_csv(input_csv, output_folder, num_cases, client)
                        break
                    else:
                        print("Por favor, ingresa un número positivo mayor que cero.")
                except ValueError:
                    logging.error("Entrada inválida. Por favor, ingresa un número entero.")
                except KeyboardInterrupt:
                    print("\nProceso interrumpido por el usuario.")
                    break
        else:
            logging.info("No se seleccionó ningún archivo de entrada. Finalizando script.")